# RNN, LSTM et Attention : traiter des données séquentielles

Nous allons voir ce qu'est un réseau de neurones récurrent, en quoi il est utile et quelles sont les applications possibles.
Face à un GROS problème des RNN, nous verrons un autre type de RNN moins naif : le Long-Short Term Memory. Enfin, face à encore une autre limitation des LSTM, nous verrons une dernière amélioration qui ouvre la porte à l'état de l'art : le mécanisme d'attention.

On arrête pas le progrès.

##1. RNN- LSTM : théorie et applications

Jusqu'à present, nous avons vu comment traiter des donnees qui se comportent bien :
  - Des tableaux de chiffres
  - Des variables catégorielles pouvant être discretisées
  - Des images, qui sont de belles matrices de chiffres

Mais comment faire avec des données un peu plus punk ?
ex: les séries temporelles, des enregistrements audio ou des textes de journaux.

Le point commun de tous ces types de données est qu'on trouve des durées variables à l'intérieur d'un même dataset: des phrases plus ou moins longues, des enregitrements de quelques secondes vs quelques minutes, etc. De plus, l'information à date $t$ dépend souvent de l'information à $t-1$: le mot d'après dépend du mot d'avant dans une phrase, le prix d'une action dépend de son prix il y a trois heures. A cette fin, on utilise un réseau de neurones récurrent, ou RNN.

###a. RNN "vanilla"

L'idée du RNN a été proposée en 1986, quand les auteurs de [1] ont réussi à réaliser la backpropagation d'un tel réseau. De nombreuses variantes existent, mais nous présentons ici un RNN relativement agnostique à toutes ces petites variations.

![rnn unfolded](https://drive.google.com/uc?id=1cJjqpuBYDjFdcq-voRzfsRslWUuftNlC)

Un RNN est donc un réseau de neurones qui prend en compte des données séquentielles. Le but est de prendre en compte le passé pour donner du sens au présent, et de prendre en compte tous les mots d'une phrase par exemple, dans la représentation finale.

Soit une séquence de données $x$=$x_1, x_2, ...x_T$. On va entrainer le même réseau $T$ fois, soit autant que de mots dans la phrase. Chaque input sera un mélange de nouvelle information (un nouveau $x_t$) et de la connaissance passée (stockée dans l'output précédent $h_t$)

Pour les états cachés, on a donc les équations suivantes (en omettant les biais) pour la passe en avant:
\begin{eqnarray}
h_t = \theta \left( W \cdot \left[ x_t, h_{t-1}\right] \right)
& =\theta \left( W_X x_t + W_H h_{t-1}\right)
\end{eqnarray}
où
- $\theta$ est $tanh$, ou n'importe quelle fonction d'activation
- W une matrice de poids
- $x_t$ l'input à date t
- $h_{t-1}$ l'état précédent du RNN

\begin{equation}
y_t = F(W_Yh_t)
\end{equation}
où
- $y_t$ est l'output du réseau pour une date $t$
- $W_Y$ est une matrice de poids
- $F()$ est une fonction d'activation, originellement juste l'identité
- $h_t$ est l'état caché du réseau pour une date $t$
 

<!---
On peut réécrire ces équations plus formellement avec :
$h_t = \begin{pmatrix} h_{t1} & h_{t2} & \dots & h_{tH} \end{pmatrix} ^\intercal$,  $y_t = \begin{pmatrix} y_{t1} & y_{t2} & \dots & y_{tN} \end{pmatrix}^\intercal$ et $x_t = \begin{pmatrix} x_{t1} & x_{t2} & \dots & x_{tI} \end{pmatrix}^\intercal$

et en passant la notation de temps en exposant pour plus de lisibilité :
$h_{k}^t = \sum_{i=1}^I w_{ik}x_i^t + \sum_{d=1}^H w_{dk}b_{d}^{t-1}$

$b_h^t = \theta_h \left( h_k^t \right)$

$y_h^t = \sum_{h'=1}^H w_{h'k}b_h^t$
--->

###b.LSTM

Un RNN basique prend donc en compte ce qui se passe l'étape d'avant pour décider de l'étape d'après. Mais souvent, on a besoin d'apprendre plus loin que l'étape précédente, et de prendre en compte tout le passé, ou du moins le plus pertinent. 

<!---
Si vous voyez un individu s'avancer vers vous avec un couteau, vous allez surement courir. Sauf si vous vous souvenez que c'est votre ami Jérémy qui vous rend le couteau de votre grand-père que vous lui aviez prété.
Mais le fait que c'est le couteau de votre grand-père a peu d'importance comparé au fait que vous avez convaincu son-sa fiancé-e de convoler avec vous la veille de leur mariage. Tout l'enjeu d'un LSTM est de se souvenir des faits les plus saillants pour obtenir les meilleures représentations des situations.
-->

Un réseau Long-Short Term Memory est un RNN avec des portes, qui gèrent l'oubli des valeurs précédentes. Dans sa version la plus courante, celle de [2], une cellule LSTM possède trois portes, que nous allons décomposer.

![lstm cell](https://drive.google.com/uc?id=1LfOd1ZkJf25tEQGo8AYqADYoYm_VNyt4)
![lstm cell](https://drive.google.com/uc?id=1Ho4pj0ybUHAUzAlyEssuF8VXc7II6vqp)

Premièrement, une "cellule" LSTM est comme une "cellule" de RNN, on lui passe un état caché et un input $x_t$ et il calcule des choses. L'état interne n'est plus l'état caché ni un simple produit suivi d'une multiplication, mais il reste simple : c'est la ligne du haut dans la cellule, $C_t^l$

Les trois portes sont trois sigmoides, qui donnent des valeurs entre 0 et 1, 0 signifiant "on laisse rien passer"/"on oublie tout" et 1 signifiant "on laisse passer toute l'information".

1. La première porte, $f_t$, s'appelle "forget gate layer" (la porte de l'oubli). Elle regarde l'état caché précédent, la nouvelle valeur, et décide de garder ou pas l'ancienne mémoire.
$$f_t = \sigma\left( W \cdot \left[ x_t, h_{t-1}\right] + b_f \right)$$
Cette équation ne vous rapelle pas quelque chose ?

2. La deuxième porte $i_t$ est l'"input gate layer"(la porte de l'entrée/la contribution). Elle décide quelles valeurs vont être modifiées/mises à jour.
En parallèle, une autre couche tanh crée de nouvelles valeurs candidates: quelles seraient les valeurs si on pouvait toutes les mettre à jour. Quand on combine les deux, cela nous donne les nouvelles valeurs à garder en mémoire.
$$i_t = \sigma \left( W_i \cdot [x_t, h_{t-1}] +b_i \right)$$
$$\tilde{C}_t = tanh \left( W_C \cdot[x_t, h_{t-1}] +b_C \right)$$

et donc :
$$\hat{C}_t = i_t * \tilde{C}_t $$

Ces deux portes apportent des modifications à l'état interne de la cellule, en décidant d'à quel point on retient le passé, et comment modifier l'état interne de la cellulle pour prendre en compte les nouvelles valeurs.

$$C_t = f_t * C_{t-1} + \hat{C}_t = f_t * C_{t-1} + i_t * \tilde{C}_t$$

3. La dernière porte est l'"output gate" (la porte de sortie). Elle ne touche plus à la mémoire de la cellule, toutes les modifications ont déjà été faites. Elle se contente de filtrer ce qu'il y a en mémoire pour donner une sortie. On prend l'état courant, que l'on fait passer par tanh (pour normaliser). En parallèle, une couche sigmoide décide de quelles valeurs on va donner en sortie. Comme pour l'input, la multiplication des 2 donne les valeurs de l'ouput.

$$o_t = \sigma \left( W_o [x_t, h_{t-1}] +b_o \right)$$
$$h_t = o_t * tanh \left( C_t \right)$$

---
<!---
Un exemple filé : prédire un mot plausible après dans la phrase "Mon frère est avocat, ma soeur est _____".
- La première porte de l'oubli permet par exemple d'oublier l'information sur le genre: nous avions un genre masculin, mais le fait d'avoir vu un nouveau sujet féminin, 'soeur', nous indique que maintenant le sujet est féminin. Quand on voit un nouveau sujet, on veut oublier le genre de l'ancien.
- La deuxième porte de l'entrée, elle, va permettre de mettre dans l'état interne (ie, en mémoire), que le sujet courant est de genre féminin.
- La porte de sortie, va elle, voir que nous avions un verbe, elle va donc surement donner de l'information concernant un nom, car souvent un nom vient apres un verbe. Il va donc donner le genre et le nombre du sujet, la conjugaison du verbe, pour accorder le nom qui vient.

Les LSTM ont été un énorme pas dans ce que l'on peut faire avec les RNN. Y a-t-il eu d'autres grands pas derrière ? Oui, il y a un consensus pour dire que le grand pas d'après a été l'attention. Ce n'est pas le seul, mais il est à la base d'un modèle qui fut état-de-l'art jusqu'en juillet 2020.
--->


###c. Méchanisme d'attention

####1. Modèle seq2seq : motivation pour l'attention

Historiquement, seq2seq est un modèle qui permet de faire de la traduction: on donne une séquence, le modèle sort une nouvelle séquence.

La mécanique basique est consitué de deux LSTM l'un après l'autre : le premier, appelé encodeur, encode toute la séquence dans le dernier état caché $h_T$. 
Le deuxième LSTM, le décodeur, prend en entrée cet encoding de la séquence et le premier mot (un token \<START\>), et sort un premier output. A chaque étape, il prend l'état caché $h'_t$ précédent et l'output qu'il vient de faire (le dernier mot prédit) pour faire une prédiction. On continue comme ça jusqu'à prédire un token de fin (\<END\>)

![lstm cell](https://drive.google.com/uc?id=13fmMDB3on4a9ZYnuYSCqpORBsxB-ESyb)

Le problème se trouve dans l'état du milieu, le fameux $c$, qui doit contenir toute l'information de la phrase. En réalité, si ce dernier output d'un LSTM est souvent suffisant pour encoder un thème, un sentiment, un topic général de la phrase, il n'est pas suffisant pour retenir toute l'information nécéssaire à une traduction. Il lui manque un élément crucial, le contexte.

####2.Attention générale

C'est là qu'apparait l'attention.
L'attention, de façon très générale, est une couche d'un réseau de neurones qui est en charge de quantifier les interdependences entre les inputs/outputs (attention générale) ou entre les inputs eux-mêmes (self-attention). Nous parlerons ici d'attention générale.

![attention mecanism](https://drive.google.com/uc?id=1uXzpMr_-G-WAhTrJf7VM4qzo2dLhTmqs)

Notation : soient $h_1, h_2..., h_T$ les états cachés du décodeur, et
$\bar{h}_1, \bar{h}_2, ..., \bar{h}_S$ les états cachés de l'encodeur. 

Le premier morceau, l'encodage, reste le même, un LSTM classique avec ses inputs, ses états cachés $h_t$.

On crée un LSTM pour le décodage. On devrait , sans attention, passer dans ce LSTM, la dernière prédiction $y_t$ et uniquement le dernier état caché $\bar{h}_t$.
Pour l'attention, on concatène au dernier état caché $\bar{h}_t$ un vecteur de contexte qui montre à quel point le dernier état caché du décodeur s'aligne avec la phrase d'origine.


On choisit d'abord un score d'alignement entre les $h$ et les $\bar{h}$. Le choix de ce score est ce qui fait la différence entre l'attention Bahdanau (ou additive) et Luong (ou multiplicative).

\begin{equation}
  score(h_t, \bar{h}_s) = \begin{cases}
      h_t^\intercal W \bar{h}_s & \text{Pour Luong/multiplicative}\\
      v_a^\intercal tanh(W_1h_t + W_2 \bar{h}_s) & \text{Pour Bahdanau/additive}
    \end{cases}       
\end{equation}

Puis on crée les poids d'attention. On utilise softmax pour rapporter le score à une distribution de probabilité (la somme de tous les poids doit faire 1).

$$ \alpha_{ts} = \frac{exp(score(h_t, \bar{h}_s))}{\sum_{s'=1}^{S} exp(score(h_t, \bar{h}_{s'}))} $$


On pondère chaque état caché de l'encodeur (donc, finalement, chaque "encoding" de chaque mot d'origine) par ces poids d'attention, et on somme. Cela donne un vecteur, issu d'une somme des mots pondérée par combien l'état caché du décodeur "colle" à chaque mot de départ.

$$c_t = \sum_{s} \alpha_{ts}\bar{h}_s $$

Finalement, on concatène ce vecteur de contexte à l'état courant du décodeur &h_t&. Cela donne un "nouveau" $h_t$, $a_t$, plus complet, qui prend en compte la coordination entre l'état courant et les mots passés dans l'encodeur. C'est sur ce vecteur que l'on ajoute une dernière couche entièrement connectée (fully connected) pour faire la prédiction d'un mot.

$$a_t = tanh(W_c[c_t;h_t])$$

Et on l'utilise comme si c'était notre état caché $h_t$ !

---

## Sources des images et références

Source de l'image RNN + détails:
https://karpathy.github.io/2015/05/21/rnn-effectiveness/

Source de l'image LSTN + détails :
http://colah.github.io/posts/2015-08-Understanding-LSTMs/



Pour plus de détails sur l'attention :
https://machinelearningmastery.com/attention-long-short-term-memory-recurrent-neural-networks/
https://blog.floydhub.com/attention-mechanism/

[1] Rumelhart, D., Hinton, G. & Williams, R. Learning representations by back-propagating errors. Nature 323, 533–536 (1986). https://doi.org/10.1038/323533a0

[2] Hochreiter, Sepp & Schmidhuber, Jürgen. (1997). Long Short-term Memory. Neural computation. 9. 1735-80. 10.1162/neco.1997.9.8.1735. 

##2. Données de travail : spam or ham ?


In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
import re
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, SimpleRNN, Dense, Embedding, LSTM, Masking, Concatenate
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import LambdaCallback, History, EarlyStopping

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import itertools

On importe les données, et on regarde rapidement leur distribution, quelques exemples pour comprendre leur format.

In [ ]:
#Import the data, check it
data = pd.read_csv("https://raw.githubusercontent.com/CMallart/ateliers-NN/main/data/spamorham/SPAM%20text%20message%2020170820%20-%20Data.csv")
data.head()
data.describe()

In [ ]:
cnt_pro = data['Category'].value_counts()
plt.figure(figsize=(12,4))
plt.bar(cnt_pro.index, cnt_pro.values, alpha=0.8, color=['blue', 'orange'])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.xticks(rotation=90)
plt.show();

Plus précisément, puisqu'on traite du language, on peut regarder quels types de phrases apparaissent dans la base. Cela sert à se donner une idée de la complexité à donner au modèle : le language est-il complexe ? A l'oeil nu, pouvons-nous faire la tache de classification ?

In [ ]:
# What kind of language is used, what kind of cleanup is necessary
def see_message(ind):
  mess=data.iloc[ind]["Message"]
  print(mess)
  return(mess)
for message in [0,15,25,35,50]:
  see_message(message)

Puisqu'on va traiter le texte et uniquement le texte, s'il existe dans les données quelques exemples sans texte, on les retire.

In [ ]:
#check if we have empty strings, or punctuation only columns
#they are not important here, we remove them
to_remove =[]
for i, row in data.iterrows():
  if re.match(r'.*[A-Za-z]+.*',row["Message"]) is None :
    print("'",row["Message"],"', at line", i)
    to_remove.append(i)
data = data.drop(to_remove, axis =0) 
print("Removed the punctuation or numbers-only lines") 

Enfin, on nettoie le texte, pour le débarasser notamment des adresses web. Cela permet de se concentrer uniquement sur le texte, et de ne pas avoir un modèle qui apprenne trop vite qu'un message avec une adresse web est forcément indésirable.

In [ ]:
#A little cleanup is necessary 
#I won't bother cleaning up the puctuation as it is done later by keras.preprocessing.tokenizer
def clean_text(text):
  #remove the https:// stuff
  text = re.sub(r"http\S+", "", text)
  #remove the urls that don't have the http:// => solution = remove all strings that have a [some non-space characters].[two or three letters]/[some non-space characters] or [some non-space characters].[two or three letters]?[some non-space characters]
  text = re.sub(r"[^\s]*\.[a-z]{2,3}(\\|\?)*[^\s]*", "", text)
  return(text)

clean_text(see_message(15))

On commence à mettre en forme les données pour entrer dans le réseau de neurones.

**Note:** l'id 0 est par défaut réservé et jamais assigné.

In [ ]:
def tokenize_and_encode(text_data):
  #fit Tokenizer : from sequences of words to lists of numbers
  tokenizer = text.Tokenizer(
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            lower=True,
            split=" ",
            oov_token="OUT_OF_VOCABULARY"
        )
  tokenizer.fit_on_texts(text_data)
  sequences = tokenizer.texts_to_sequences(text_data)
  print("Example of output : ",  sequences[2])
  print(" For the input : ",text_data[2])

  #pad the data for input into the RNN 
  #maxlen = max([len(x) for x in sequences])
  padded_sequences = sequence.pad_sequences(sequences, padding="post", maxlen=50)

  print('Number of Unique Tokens: %d' % len(tokenizer.word_index))
  return(padded_sequences, len(tokenizer.word_index)+1)

On encode les données et les targets. Les targets sont one-hot-encodées.

In [ ]:
#encoding the whole dataset

#targets
#Binarize categories
targets = data["Category"].apply(lambda x: 0 if x=="spam" else 1).tolist()
one_hot_targets = np.eye(2)[targets]

#text
text_data, vocab_size = tokenize_and_encode(data["Message"].apply(clean_text))

On sépare la base en base d'entrainement et base de test, et on vérifie que tout est au bon format.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text_data, one_hot_targets, test_size=0.33, random_state=17)

#Convert those arrays to numpy as it is the desired input to the Keras RNN
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print("Example of encoded train sequence: ", X_train[0])
print("Example of encoded target: ", y_train[0])

In [ ]:
timestep = X_train[0].shape[0] # the length of a sequence (sentence, or time series, etc) is the timestep. It can vary, but we chose to pad to have a fixed length. Why ?
n_features = 1 #each input on each timestep only is one number, so it is of size 1

###Question 1 
* Qu'avez-vous constaté comme phénomènes dans les données ?
* Que fait la fonction tokenize_and_encode ?
* Pourquoi dans le code de tokenize_and_encode, utilise-t-on keras.preprocessing.pad_sequences ?

On définit pour aller plus vite dans l'exploration graphique des résulats deux fonction : plot_history et plot_confusion_matrix. Il suffira de les évoquer après l'entrainement.

In [ ]:
#Fonction outil pour la suite
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
## multiclass or binary report
## If binary (sigmoid output), set binary parameter to True
def full_multiclass_report(model,
                           x,
                           y_true,
                           classes,
                           batch_size=32,
                           binary=False):
    
    # 2. Predict probabilities and stores in y_pred
    y_pred = model.predict(x, batch_size=batch_size)
    y_pred = y_pred.argmax(axis=1)
    y_true = y_true.argmax(axis=1)
    
    # 3. Print accuracy score
    print("Accuracy : "+ str(accuracy_score(y_true,y_pred)))
    
    print("")
    
    # 4. Print classification report
    print("Classification Report")
    print(classification_report(y_true,y_pred,digits=5))    
    
    # 5. Plot confusion matrix
    cnf_matrix = confusion_matrix(y_true, y_pred)
    print(cnf_matrix)
    plot_confusion_matrix(cnf_matrix,classes=classes)

##3. Mon premier RNN

Il s'agit ici de l'architecture de base d'un RNN comme vu précédemment.
On ajoute une couche d'embedding, puis une couche SimpleRNN, qui est la couche RNN de base. Cette couche ne va retourner que le dernier état caché. Enfin, on ajoute une couche d'output complètement connectée.



In [ ]:
epochs = 25
batch_size = 16
lr = 0.5

In [ ]:
# Keras' input for any recurrent neural network requires the following shape : (batch_size, timestep, n_features)
#batch_size can be None, it will be input at train time, this is a default in Keras if all timesteps are equal
#timestep is the length of your sequence
#n_features is the number of features on each timestep. Had you done some embedding pre-processing, it would be larger than 1
input=Input(shape=(timestep))
embedding = Embedding(input_dim=vocab_size, output_dim=200, mask_zero=True, embeddings_initializer="zeros")(input)
rnn_layer = SimpleRNN(units = 24, activation = "sigmoid", recurrent_initializer= "random_normal")(embedding) #in : size (None, timesteps, 64), out : size (None, units)
dense_layer=Dense(2, activation = "softmax")(rnn_layer) #in : size(None, units), out : size(None,1)
model_rnn=Model(input,dense_layer)

On compile le modèle avec un SGD, stochastic gradient descend, qui est lui aussi la version de base pour les stratégies d'optimisation.

In [ ]:
#Output the model skeleton, and compile
model_rnn.summary()

optim = SGD(learning_rate=lr)
model_rnn.compile(optimizer=optim,loss="binary_crossentropy", metrics = ["binary_accuracy"])

#print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[1].get_weights())) #affiche les poids de la couche n*1, cad la couche SimpleRNN
history = History()

In [ ]:
history=model_rnn.fit(X_train, y_train, epochs =epochs, batch_size=batch_size, verbose = 1, validation_split=0.2, callbacks = [history])

On regarde les résulats, notamment les courbes de loss et d'accuracy, ainsi que la matrice de confusion.

In [ ]:
plot_history(history)

In [ ]:
full_multiclass_report(model_rnn, X_test, y_test, [0,1])

### Question 2

*    A quoi sert la couche d'embedding ?
*    Quand considérez-vous qu'il y a du sur-apprentissage (overfitting) ?


### Question 3: 
C'est beau comme modèle  ! Maintenant, enlevez de votre couche d'embedding le paramètre `mask=True`. Oh-oh...
*    Quel comportment de votre RNN pendant l'entrainement vous met la puce à l'oreille que quelque chose ne va pas ? 

Décommenter la ligne `#print_weights`, et relancer l'entrainement avec le callback print_weights.

Ce problème est un problème courant des réseaux de neurones récurrents (mais pas que !), il s'appelle le <font color='red'>Vanishing Gradient</font>, ou la disparition du gradient. Plus de détails ici :
https://towardsdatascience.com/the-vanishing-gradient-problem-69bf08b15484

*   Pouvez-vous expliquer simplement le phénomène ? 

##4. LSTM, évolution du RNN

Des solutions au gradient qui disparait sont:
  - changer de fonction d'activation, sigmoid est bien identifié pour poser des problèmes. (pourquoi ?)
  - bien initialiser les poids (pourquoi ?)
  - ajouter des couches de normalisation (pourquoi ?)
  - avec tensorflow, masquer proprement (nous venons de le voir)
Dans le cas inverse, un gradient qui explose, souvent, on clippe le gradient.

Le LSTM résout plusieurs problèmes liées aux RNN de base, mais surtout celui du gradient qui disparait ou explose. Il permet aussi plus de finesse dans ce dont on se "souvient" ou pas dans le réseau.

---

Nous allons reprendre notre problème initial de classification de messages, mais maintenant avec un LSTM, et des fonctions d'activation plus fines.

Pour le LSTM, la documentation Keras est bien faite :
https://keras.io/api/layers/recurrent_layers/lstm/ et 
https://keras.io/api/layers/activations/

---

Attention, le choix par défaut est quand même un choix ! Soyez au moins conscients d'avoir choisi telle fonction d'activation ou telle initialisation, car nous avons vu que cela peut avoir de grosses conséquences. 

In [ ]:
#hyperparametrees
epochs = 20
batch_size = 128
lr = 0.04

In [ ]:
#architecture du modele
input=Input(shape=(timestep))
embedding = Embedding(input_dim=vocab_size, output_dim=200, mask_zero=True)(input)
lstm_layer = LSTM(units = 64, recurrent_activation = "relu")(embedding)
dense_layer = Dense(2, activation = "softmax")(lstm_layer)

model_lstm=Model(input,dense_layer)

On choisit ici un optimiseur un peu différent : Adam. Il est plus adapté à des opération complexes, en partie car il permet au learning rate de s'adapter. 

In [ ]:
#compilation
from keras.utils import plot_model
plot_model(model_lstm)

optim = Adam(learning_rate=lr)
model_lstm.compile(optimizer=optim,loss="binary_crossentropy", metrics = ["binary_accuracy"])

history = History()

In [ ]:
history=model_lstm.fit(X_train, y_train, epochs =epochs, batch_size=batch_size, verbose = 1, validation_split=0.2, callbacks = [history])

In [ ]:
plot_history(history)
full_multiclass_report(model_lstm, X_test, y_test, [0,1])

###Question 5

Retirez `mask=True`  la couche d'embedding pour ce modèle. Y a-t-il toujours des problèmes ? Arrivent-ils toujours aussi vite ?

##5. Attention

Cette attention est un many-to-one, prenant donc les outputs d'un LSTM et applicant l'attention pour obtenir seulement le dernier output, car nous faisons de la classification. Pour de la traduction par exemple, la couche serait différente.

Code de l'attention inspiré de ce blog :
https://matthewmcateer.me/blog/getting-started-with-attention-for-classification/
et de ce repository GitHub :
https://github.com/philipperemy/keras-attention-mechanism/blob/master/attention/attention.py


Keras possède aussi un layer d'Attention selon Luong et d'attention selon Badanau. https://keras.io/api/layers/attention_layers/additive_attention/. Nous ne l'utilisons pas pour voir que les couches Keras sont des opérations mathématiques à part entière, et qu'on peut faire toutes les opérations décrites dans les équations précédentes avec quelques couches Keras. A retenir : savoir à quoi correspondent les couches et ce que l'on met dans son modèle est plus important que d'empiler des couches aveuglément et d'avoir plein de puissance de calcul.

In [ ]:
#hyperparametres
epochs = 20
batch_size = 128
lr = 0.04

In [ ]:
#Modèle avec attention type Luong

input=Input(shape=(timestep))
embedding = Embedding(input_dim=vocab_size, output_dim=200, mask_zero=True)(input)
(lstm_seq, h_t, state_c) = LSTM(units = 128, recurrent_activation = "relu", return_sequences=True, return_state=True)(embedding)

#on calcule le score pour h_t
units = int(h_t.shape[1])
score_first_part = Dense(units, name="attention_score_part_1")(lstm_seq) #that is the W*h_s_bar in the slides' equations
score =  dot([score_first_part, h_t], [2, 1], name='attention_score') # that is the h_t*W*h_s_bar in the slides' equations

#on calcule les poids d'attention
attention_weights = Activation("softmax", name="attention_weights")(score)
  
#le vecteur de contexte est le produit scalaire des vecteur d'attention et des états cachés du LSTM
context_vector = dot([attention_weights,lstm_seq], [1, 1], name ="context_vector")

#on crée le vecteur d'attention en concetenant context et h_t, et en l'activant pat tanh
concat_att_state= concatenate([context_vector, h_t])
attention_vector = Dense(64, use_bias=False, activation='tanh', name='attention_vector')(concat_att_state)

dense_layer = Dense(2, activation = "relu", name="output")(attention_vector)
model_att = Model(inputs=input, outputs=dense_layer)


Nous utilisons la version fonctionelle de Keras plutot que sa version séquentielle, ce que vous avez du remarquer dans ce TP. 
Cela permet d'aller chercher individuellement les couches et leurs outputs, de recycler des outputs, et de faire plusieurs outputs pour un même modèle.
Cela permet donc de pouvoir utiliser `lstm_seq`, qui n'est autre que la séquence $\bar{h}_1, \bar{h}_2,...,\bar{h}_S$, autant de fois que nous avons besoin.

In [ ]:
from keras.utils import plot_model
plot_model(model_att)

In [ ]:
model_att.summary()

Nous allons visulaiser l'attention. Pour cela, il nous faut obtenir les poids d'attention. 
Puisque nous utilisons la version fonctionelle de Keras, nous pouvons réaliser un petit tour de passe-passe. Nous créons un modèle `a_w_model` pour attention weight model, que nous initialisons avec les mêmes couches que le modèle `model_att`. Ce n'est pas seulement les mêmes noms, ce sont les mêmes couches avec les mêmes matrices de poids, partagées entre les deux modèles ! 

De cette façon, entrainer `model_att` va faire varier identiquement les poids pour `a_w_model`. L'output de `a_w_model` étant la layer "attention_weights" de `model_att`, il suffit de prédire quelque chose avec `a_w_model` pour obtenir les poids d'attention.


In [ ]:
#model for attention weights, share the exact same layers as our main model
a_w_model = Model(inputs= model_att.inputs, outputs = model_att.get_layer("attention_weights").output)

Nous définissons un callback, pour pouvoir visualiser les poids d'attention à la fin de chaque époque et voir comment ils varient.

In [ ]:
from tensorflow.keras.callbacks import Callback

# to visualise, we use the great boutny of keras' functional model.
#we create our model that gets trained. And then on the side, we create another model, that shares exactly the same layers, but stops at the attention layer
#the first model will train, and because they share layers, the layers of the visu model will also train
#it suffices to get the output of the visu model to get the attention weights of the trained model

class VisualiseAttentionMap(Callback):
    def __init__(self, attention_weights_model, x_test, max_epochs, output_dir=None):
        super(Callback, self).__init__()
        self.visu_model = attention_weights_model
        # best_weights to store the weights at which the minimum loss occurs.
        self.x_test = x_test
        self.output_dir = output_dir
        self.max_epoch = max_epochs

    def on_epoch_end(self, epoch, logs=None):
        attention_map = self.visu_model.predict(self.x_test)
        #print(attetion_map.shape)
        # top is attention map.
        # bottom is ground truth.
        #plt.imshow(np.concatenate([attention_map, x_test_mask]), cmap='hot')

        plt.imshow(attention_map, cmap='hot')

        iteration_no = str(epoch).zfill(3)
        plt.axis('off')
        plt.title(f'Iteration {iteration_no} / {self.max_epoch}')
        if self.output_dir is not None :
          if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
          plt.savefig(f'{output_dir}/epoch_{iteration_no}.png')
        plt.show()

On compile, on définit les callbacks. 

In [ ]:
optim = Adam(learning_rate =lr, clipnorm=1)
model_att.compile(optimizer=optim,loss="binary_crossentropy", metrics = ["binary_accuracy"])

history = History()
#earlyStop = EarlyStopping(monitor = "val_binary_accuracy", patience = 4, restore_best_weights=True)
attmap = VisualiseAttentionMap(a_w_model, X_train[0:10], epochs)

In [ ]:
model_att.fit(X_train, y_train, epochs =epochs, batch_size=128, verbose = 1, validation_split=0.2, 
              callbacks =[history, attmap])

In [ ]:
plot_history(history)
full_multiclass_report(model_att, X_test, y_test, [0,1])

### Question 6

Faites tourner le modèle avec attention sur 20 époques. 
*   Que constatez-vous au niveau des résultats et de la rapidité de convergence ? Est-ce qu'on sur-apprend ?

Rajouter l’early stopping dans le modèle.

*   Qu’a permis de faire l’attention dans ce modèle ? 

*   Que permet de faire le Callback que nous avons défini, VisualAttentionMap ? Qu’observez-vous au niveau des outputs de VisualAttentionMap ? 



###Question 7

A votre avis, l'attention est-elle nécéssaire dans notre cas de figure, avec spam-ou-ham ? Si oui, pourquoi, si non, pourquoi pas ?